<a href="https://colab.research.google.com/github/rishikksh20/melgan/blob/master/melgan_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rishikksh20/melgan.git

Cloning into 'melgan'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 142 (delta 71), reused 97 (delta 35), pack-reused 0
Receiving objects: 100% (142/142), 293.44 KiB | 541.00 KiB/s, done.
Resolving deltas: 100% (71/71), done.


In [2]:
# For Full-Band MelGAN
!gdown https://drive.google.com/uc?id=1Pl-RnCnHTSqYvkZ-VoP_pz2TKOG5rcID # https://drive.google.com/file/d/1Pl-RnCnHTSqYvkZ-VoP_pz2TKOG5rcID/view?usp=sharing

# For Multi-Band MelGAN
# !gdown https://drive.google.com/uc?id=1Go2NB2_cfwhyb7jMwnczmyPZAbdUM9ex # https://drive.google.com/file/d/1Go2NB2_cfwhyb7jMwnczmyPZAbdUM9ex/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1Pl-RnCnHTSqYvkZ-VoP_pz2TKOG5rcID
To: /content/second_9323377_0930.pt
109MB [00:00, 163MB/s]  


In [3]:
!pip install -r melgan/requirements.txt

     |████████████████████████████████| 317kB 2.7MB/s 


In [12]:
!pip install unidecode

     |████████████████████████████████| 245kB 2.7MB/s 


In [4]:
%cd melgan/

/content/melgan


In [7]:
import torch
import numpy as np
from model.generator import Generator
from utils.hparams import HParam, load_hparam_str
from utils.pqmf import PQMF

In [ ]:
# Load MelGAN

In [9]:
checkpoint = torch.load('/content/second_9323377_0930.pt')
hp = load_hparam_str(checkpoint['hp_str'])

vocoder = Generator(hp.audio.n_mel_channels, hp.model.n_residual_layers,
                        ratios=hp.model.generator_ratio, mult = hp.model.mult,
                        out_band = hp.model.out_channels).cuda()
vocoder.load_state_dict(checkpoint['model_g'])
vocoder.eval(inference=False)

In [38]:
text = "To improve the alignment accuracy and thus reduce the information gap."

In [39]:
# Load Tacotron

In [40]:
tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2').cuda()
# preprocessing
sequence = np.array(tacotron2.text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)

# run the models
with torch.no_grad():
    _, mel, _, _ = tacotron2.infer(sequence)

Using cache found in /root/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub


In [41]:
# MelGAN

In [42]:
MAX_WAV_VALUE = 32768.0
with torch.no_grad():
    mel = mel.detach()
    if len(mel.shape) == 2:
        mel = mel.unsqueeze(0)
    mel = mel.cuda()
    audio = vocoder.inference(mel)
    # For multi-band inference
    if hp.model.out_channels > 1:
        pqmf = PQMF()
        audio = pqmf.synthesis(audio).view(-1)
    
    audio = audio.squeeze() # collapse all dimension except time axis
    audio = audio[:-(hp.audio.hop_length*10)]
    audio = MAX_WAV_VALUE * audio
    audio = audio.clamp(min=-MAX_WAV_VALUE, max=MAX_WAV_VALUE-1)
    audio = audio.short()
    audio = audio.cpu().detach().numpy()

In [43]:
from IPython.display import Audio
Audio(audio, rate = hp.audio.sampling_rate)

In [22]:
# WaveGlow

In [ ]:
waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

In [44]:
# run the models
with torch.no_grad():
  audio = waveglow.infer(mel)
audio = audio[0].data.cpu().numpy()
Audio(audio, rate = hp.audio.sampling_rate)